# UAE Rental Price Prediction
## Goal
 Build and end-to-end Machine Learning Regression Pipeline to predict **annual rental prices (rent)** of UAE properties using real-world real estate listing data. 

## Dataset Overview
- rows: 73,742
- columns: 17
- Target Variable: Rent (annual rent in AED)
- Problem Type: Supervised Regression
- Evaluation Metric: RMSE 

## Key Constraints
- Prevent data leakage
- Use reproducible preprocessing pipelines 
- Establish a strong baseline before complex models 

In [2]:
import pandas as pd
df=pd.read_csv("data/dubai_properties.csv")


## 1. Initial Data inspection

In [3]:
df.head()

,Address,Rent,Beds,Baths,Type,Area_in_sqft,Rent_per_sqft,Rent_category,Frequency,Furnishing,Purpose,Posted_date,Age_of_listing_in_days,Location,City,Latitude,Longitude
0,"The Gate Tower 2, The Gate Tower, Shams Gate D...",124000,3,4,Apartment,1785,69.467787,Medium,Yearly,Unfurnished,For Rent,2024-03-07,45,Al Reem Island,Abu Dhabi,24.493598,54.407841
1,"Water's Edge, Yas Island, Abu Dhabi",140000,3,4,Apartment,1422,98.452883,Medium,Yearly,Unfurnished,For Rent,2024-03-08,44,Yas Island,Abu Dhabi,24.494022,54.607372
2,"Al Raha Lofts, Al Raha Beach, Abu Dhabi",99000,2,3,Apartment,1314,75.342466,Medium,Yearly,Furnished,For Rent,2024-03-21,31,Al Raha Beach,Abu Dhabi,24.485931,54.600939
3,"Marina Heights, Marina Square, Al Reem Island,...",220000,3,4,Penthouse,3843,57.246942,High,Yearly,Unfurnished,For Rent,2024-02-24,57,Al Reem Island,Abu Dhabi,24.493598,54.407841
4,"West Yas, Yas Island, Abu Dhabi",350000,5,7,Villa,6860,51.020408,High,Yearly,Unfurnished,For Rent,2024-02-16,65,Yas Island,Abu Dhabi,24.494022,54.607372


## 2. Data types and missing values 

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73742 entries, 0 to 73741
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Address                 73742 non-null  object 
 1   Rent                    73742 non-null  int64  
 2   Beds                    73742 non-null  int64  
 3   Baths                   73742 non-null  int64  
 4   Type                    73742 non-null  object 
 5   Area_in_sqft            73742 non-null  int64  
 6   Rent_per_sqft           73742 non-null  float64
 7   Rent_category           73742 non-null  object 
 8   Frequency               73742 non-null  object 
 9   Furnishing              73742 non-null  object 
 10  Purpose                 73742 non-null  object 
 11  Posted_date             73742 non-null  object 
 12  Age_of_listing_in_days  73742 non-null  int64  
 13  Location                73742 non-null  object 
 14  City                    73742 non-null

In [5]:
#Percentage of missing values in each column
missing_pct = (df.isnull().mean()*100).sort_values(ascending=False)
missing_pct

Longitude                 0.975021
Latitude                  0.975021
Address                   0.000000
Rent                      0.000000
Beds                      0.000000
Area_in_sqft              0.000000
Rent_per_sqft             0.000000
Baths                     0.000000
Type                      0.000000
Frequency                 0.000000
Rent_category             0.000000
Furnishing                0.000000
Purpose                   0.000000
Age_of_listing_in_days    0.000000
Posted_date               0.000000
City                      0.000000
Location                  0.000000
dtype: float64

## 3. Data Leakage Prevention
Certain columns are derived directly from the target variable (Rent) or contain redundant information, and can cause data leakages. These columns are removed to ensure a fair evaluation.


In [6]:
leakage_and_useless_cols=["Rent_per_sqft","Rent_category","Frequency","Purpose"]
df=df.drop(columns=leakage_and_useless_cols, errors="ignore")

## 4. Feature-Target Seperation (Define features and targets)

In [7]:
y=df["Rent"]
X=df.drop(columns=["Rent"])

#note: X contains only information available before prediction time    

## 5. Train-Validation Split

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)
X_train.shape, X_val.shape

((58993, 12), (14749, 12))

The dataset is split into training (80%) and validation (20%) sets to simulate unseen data and evaluate generalization performance.
A fixed random seed ensures reproducibility of results.
A fixed random seed is used to ensure reproducibility of the train–validation split, allowing consistent evaluation and comparison of model performance.


## 6. Feature type Identification 
Numerical and categorical features are identified to enable seperate preprocessing strategies.
 Numerical features will be imputed (fill in missing values in a smart, consistent way) using median values, while categorical features will be imputed and one-hot encoded (converts categories into numbers without creating fake order of data)

In [9]:
#tells us which is columns are numerical and which categorical from the feature matrix X
num_features=X.select_dtypes(include=["int64","float64"]).columns
cat_features=X.select_dtypes(include=["object"]).columns

num_features, cat_features

#Note: 
# Imputation: Replacing missing values using statistics learned from training data
#One-hot encoding: Representing categorical values as binary columns so models don’t assume order

(Index(['Beds', 'Baths', 'Area_in_sqft', 'Age_of_listing_in_days', 'Latitude',
        'Longitude'],
       dtype='object'),
 Index(['Address', 'Type', 'Furnishing', 'Posted_date', 'Location', 'City'], dtype='object'))

Although the Address field contains rich location information, it is excluded from the baseline model due to its high cardinality and unstructured nature.
Instead, structured location features such as City, Location, Latitude, and Longitude are used.
Address-level features may be explored in later feature engineering stages.


In [10]:
# Drop high-cardinality text feature for baseline
X = X.drop(columns=["Address"], errors="ignore")

# Re-identify feature types after drop
num_features = X.select_dtypes(include=["int64", "float64"]).columns
cat_features = X.select_dtypes(include=["object"]).columns

num_features, cat_features


(Index(['Beds', 'Baths', 'Area_in_sqft', 'Age_of_listing_in_days', 'Latitude',
        'Longitude'],
       dtype='object'),
 Index(['Type', 'Furnishing', 'Posted_date', 'Location', 'City'], dtype='object'))

## 7. Preprocessing Pipeline
A preprocessing pipeline is constructed to ensure that all numerical and categorial features are transformed correctly and without data leakage.
Numerical features are imputed using median data, while categorical features are imputed and one-hot encoded.

In [11]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer    

In [12]:
#Numerical feature pipeline
num_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median"))
])


#Categorical feature pipeline
cat_pipeline = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])
#combine pipelines
preprocessor = ColumnTransformer(transformers=[
    ("num", num_pipeline, num_features),
    ("cat", cat_pipeline, cat_features)
])

##8. Baseline Model: Linear Regression
A linear regression model is used as a baseline to establish a performance benchmark.
Every other model must outperform this model to justify additional complexity.

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


In [14]:
#Full pipeline:  preprocessing + model
lin_reg_model = Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", LinearRegression())
])

## 9. Model Training


In [15]:
lin_reg_model.fit(X_train,y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

## 10. Baseline Evaluation (RMSE)



The trained pipeline is used to generate rental price predictions for the validation set.
Step 1 applies the same preprocessing transformations learned from the training data and produces predicted rent values for unseen properties.

Model performance is evaluated on the validation set using Root Mean Squared Error (RMSE), which measures the average deviation between predicted and actual rental prices.


In [16]:
import numpy as np
# 1. predict on validation set
y_val_pred= lin_reg_model.predict(X_val)

#2. compute RMSE 
mse= mean_squared_error(y_val,y_val_pred)
baseline_rmse=np.sqrt(mse)
baseline_rmse



np.float64(474880.37892009143)

The baseline Linear Regression model achieved an RMSE of 474885.241634566 AED on the validation set.
This score serves as a reference point for evaluating more complex models.

In [17]:
y.min(),y.max(),y.median(),y.mean()

(np.int64(0),
 np.int64(55000000),
 np.float64(98000.0),
 np.float64(147925.00166797754))

In [18]:
y.describe()

count    7.374200e+04
mean     1.479250e+05
std      3.069658e+05
min      0.000000e+00
25%      5.499900e+04
50%      9.800000e+04
75%      1.700000e+05
max      5.500000e+07
Name: Rent, dtype: float64

## 11. Target Transformation
Due to extreme right-skew and presence of high-value outliers, a logarithmic transformation is applied to the target variable.
This stabilizes variance and allows the model to focus on relative errors rather than absolute magnitude. 

In [19]:
import numpy as np
y_log=np.log1p(y)

In [20]:
#redoing the split using same X 
X_train, X_val, y_train_log, y_val_log = train_test_split(
    X,y_log, test_size=0.2, random_state=42
)

In [21]:
lin_reg_model.fit(X_train, y_train_log)


,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [22]:
y_val_log_pred = lin_reg_model.predict(X_val)

rmse_log= np.sqrt(mean_squared_error(y_val_log,y_val_log_pred))
rmse_log

np.float64(0.3244781056986314)

The model is re-trained in logarithmic-transformed target and evaluated using RMSE in log space. This provides a more robust assessment under heavy/extreme skews 

In [23]:
#converting predictions back to AED to make it interpretable 
y_val_pred_original=np.expm1(y_val_log_pred)
rmse_original = np.sqrt(mean_squared_error(y_val,y_val_pred_original))

rmse_original

np.float64(375857.3318910722)

We apply a log transformation to the rental prices to stabilize variance and reduce the impact of extreme outliers, train the model in log space, and then reverse the transformation to interpret predictions in original AED units.

## 12. Model Improvement Strategy

The baseline and log-transformed linear models establish a reliable performance reference but remain limited by linear assumptions.
In this phase, more expressive models are explored to capture non-linear relationships and feature interactions inherent in real estate pricing.
Model complexity is increased in a controlled manner, and all improvements are evaluated against previous benchmarks.

## 13. Ridge Regression
This is introduced to address multicollinearity and overfitting caused by high-dimensional one-hot encoded features.
L2 regularization stabilizes coefficient estimates and often improves generalization performance in tabular datasets. 

In [24]:
from sklearn.linear_model import Ridge

In [25]:
ridge_model= Pipeline(steps=[
    ("preprocess", preprocessor),
    ("model", Ridge(alpha=1.0))
])

In [26]:
ridge_model.fit(X_train, y_train_log)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [27]:
y_val_log_pred_ridge=ridge_model.predict(X_val)
rmse_log_ridge = np.sqrt(mean_squared_error(y_val_log,y_val_log_pred_ridge))
rmse_log_ridge

np.float64(0.47162310809554686)

In [28]:
#convert back to AED and compute RMSE
y_val_pred_ridge= np.expm1(y_val_log_pred_ridge)
rmse_ridge_aed=np.sqrt(mean_squared_error(y_val,y_val_pred_ridge))
rmse_ridge_aed

np.float64(319589.1449363688)

In [29]:
results = {
    "Linear (log)": rmse_original,
    "Ridge (log)": rmse_ridge_aed
}
results

{'Linear (log)': np.float64(375857.3318910722),
 'Ridge (log)': np.float64(319589.1449363688)}

Ridge regression significantly reduced RMSE in original AED space compared to log-linear baseline, indicating improved robustness to extreme rental values.
although log-space RMSE increased slightly, the reduction in large absolute errors make ridge a stronger candidate for real-world pricing tasks.

## 14. Random Forest Regression
Random forest is introduced to capture non-linear relationships and feature interactions that linear models cannot represent.
Ensemble tree-based models are particularly effective for tabular real estate data, where pricing depends on complex interactions between location, size and property attributes. 

In [30]:
from sklearn.ensemble import RandomForestRegressor

In [31]:
rf_model=Pipeline(steps=[
    ("preprocess",preprocessor),
    ("model",RandomForestRegressor(
        n_estimators=200,
        max_depth=None,
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    ))
])

In [32]:
rf_model.fit(X_train, y_train_log)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [33]:
y_val_log_pred_rf=rf_model.predict(X_val)
rmse_log_rf=np.sqrt(mean_squared_error(y_val_log,y_val_log_pred_rf))
rmse_log_rf

np.float64(0.2687281065653738)

In [34]:
y_val_pred_rf=np.expm1(y_val_log_pred_rf)
rmse_rf_aed=np.sqrt(mean_squared_error(y_val,y_val_pred_rf))
rmse_rf_aed

np.float64(464223.19218706386)

In [35]:
results = {
    "Linear (log)": rmse_original,
    "Ridge (log)": rmse_ridge_aed,
    "Random Forest (log)": rmse_rf_aed
}

results


{'Linear (log)': np.float64(375857.3318910722),
 'Ridge (log)': np.float64(319589.1449363688),
 'Random Forest (log)': np.float64(464223.19218706386)}

Although Random Forest improved performance in log space, it performed poorly after back-transformation to the original scale.
This is attributed to the interaction between high-cardinality one-hot encoded features and tree-based splitting, which led to unstable predictions for extreme values.
As a result, Gradient Boosting models are explored as a more suitable non-linear alternative.


## 15. Gradient Boosting Regression

Gradient Boosting is explored to model non-linear relationships and complex feature interactions while maintaining stability on high-dimensional one-hot encoded data.
Unlike Random Forests, boosting learns additive corrections, often resulting in better generalization for skewed real estate pricing data.


In [36]:
from sklearn.ensemble import GradientBoostingRegressor

In [37]:
gbr_model= Pipeline(steps=[
    ("preprocess",preprocessor),
    ("model",GradientBoostingRegressor(
        n_estimators=300,
        learning_rate=0.05,
        max_depth=3,
        random_state=42
    ))


])

In [38]:
gbr_model.fit(X_train,y_train_log)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocess', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers cont

In [39]:
y_val_log_pred_gbr= gbr_model.predict(X_val)
rmse_log_gbr= np.sqrt(mean_squared_error(y_val_log, y_val_log_pred_gbr))
rmse_log_gbr

np.float64(0.31235719218202584)

In [40]:
y_val_pred_gbr=np.expm1(y_val_log_pred_gbr)
rmse_gbr_aed=np.sqrt(mean_squared_error(y_val,y_val_pred_gbr))
rmse_gbr_aed

np.float64(481782.2173352426)

In [41]:
results = {
    "Linear (log)": rmse_original,
    "Ridge (log)": rmse_ridge_aed,
    "Random Forest (log)": rmse_rf_aed,
    "Gradient Boosting (log)": rmse_gbr_aed
}

results


{'Linear (log)': np.float64(375857.3318910722),
 'Ridge (log)': np.float64(319589.1449363688),
 'Random Forest (log)': np.float64(464223.19218706386),
 'Gradient Boosting (log)': np.float64(481782.2173352426)}

In [42]:
{'Linear (log)': np.float64(375857.3318910722),
 'Ridge (log)': np.float64(319589.1449363688),
 'Random Forest (log)': np.float64(464223.19218706386),
 'Gradient Boosting (log)': np.float64(481782.2173352426)}

{'Linear (log)': np.float64(375857.3318910722),
 'Ridge (log)': np.float64(319589.1449363688),
 'Random Forest (log)': np.float64(464223.19218706386),
 'Gradient Boosting (log)': np.float64(481782.2173352426)}

Although Gradient Boosting achieved competitive performance in log space, it performed poorly after back-transformation to the original AED scale.
This indicates instability when combining tree-based models, high-cardinality one-hot encoded features, and log-transformed targets.
As a result, Ridge Regression is selected as the final model due to its superior stability and lower absolute error.


## 16. Error Analysis
To better understand model behavior, prediction errors are analyzed across different rental price ranges.
This helps identify systematic weaknesses, assess model stability, and guide any final feature-level refinements.

In [45]:
#7.1 Compute Residuals [Residual = actual - predicted]
#final model predictions (Ridge is our chosen model)
y_val_pred_final= y_val_pred_ridge
residuals=y_val-y_val_pred_final

#positive residual -> model underestimated
#negative residual -> model overestimated

In [ ]:
#7.2 - Error by price bucket

error_df = pd.DataFrame({
    "actual_rent": y_val,
    "predicted_rent": y_val_pred_final,
    "abs_error": np.abs(residuals)  
})

#create price buckets based on actual rent 
error_df["rent_bucket"] = pd.qcut( #qcut = quantile cut 
    error_df["actual_rent"],
    q=4,
    labels = ["Low","Mid-Low","Mid-High","High"]
)
error_df.groupby("rent_bucket")["abs_error"].mean()

C:\Users\DELL\AppData\Local\Temp\ipykernel_17780\623281516.py:15: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  error_df.groupby("rent_bucket")["abs_error"].mean()


rent_bucket
Low          14343.834442
Mid-Low      27630.290595
Mid-High     42475.110681
High        175310.705164
Name: abs_error, dtype: float64

In [50]:
#7.3 Relative error
error_df["relative_error"] = error_df["abs_error"] / error_df["actual_rent"]

error_df.groupby("rent_bucket")["relative_error"].mean()

C:\Users\DELL\AppData\Local\Temp\ipykernel_17780\3363909303.py:4: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  error_df.groupby("rent_bucket")["relative_error"].mean()


rent_bucket
Low              inf
Mid-Low     0.353561
Mid-High    0.319214
High        0.386840
Name: relative_error, dtype: float64

In [52]:
#7.4 Check Suspicious cases
error_df.sort_values("abs_error", ascending = False).head(10)

#inspecting worst predictions, eg. very high rent, rare locations, possibly noisy listings

,actual_rent,predicted_rent,abs_error,rent_bucket,relative_error
56785,55000000,2.802735e+07,2.697265e+07,High,0.490412
58134,4600000,1.915796e+07,1.455796e+07,High,3.164774
31677,16000000,6.698405e+06,9.301595e+06,High,0.581350
23943,6500000,5.459721e+04,6.445403e+06,High,0.991600
55471,12000000,7.458402e+06,4.541598e+06,High,0.378466
59484,800000,4.862972e+06,4.062972e+06,High,5.078715
21749,4500000,5.620127e+05,3.937987e+06,High,0.875108
60733,5500000,1.800458e+06,3.699542e+06,High,0.672644
48102,4499999,8.374801e+05,3.662519e+06,High,0.813893
61705,4500000,1.119185e+06,3.380815e+06,High,0.751292


Error analysis shows that model performs reliably across low and mid range rental properties, while large absolute errors are primarily driven by extreme high-value listings.
Given the inherent variability and sparsity of luxury rental data, Ridge regression offers the most stable and generalizable solution among the models evaluated 


## 17. Final Model Selection

After evaluating multiple modeling approaches, including Linear Regression, Ridge Regression, Random Forest, and Gradient Boosting, Ridge Regression trained on the log-transformed target achieved the best performance in terms of RMSE in original AED scale.

Tree-based models underperformed due to instability introduced by high-cardinality one-hot encoded features combined with exponential back-transformation.

Ridge Regression offered the best balance between stability, generalization, and interpretability, making it the final selected model.


## 18. Cross-Validation

To ensure that model performance is not dependent on a single train–validation split, 5-fold cross-validation is performed using Ridge Regression on the log-transformed target.

This provides a more reliable estimate of generalization performance and measures stability across multiple data splits.


In [56]:
from sklearn.model_selection import KFold, cross_val_score

kf=KFold(n_splits=5, shuffle=True, random_state=42)

In [57]:
cv_scores= cross_val_score(
    ridge_model,
    X,
    y_log,
    scoring="neg_root_mean_squared_error",
    cv=kf,
    n_jobs=-1
)

In [58]:
cv_rmse_log = -cv_scores

cv_rmse_log.mean(), cv_rmse_log.std()


(np.float64(0.4799209643503229), np.float64(0.013696860896642073))

In [59]:
import numpy as np

approx_multiplier_error = np.exp(cv_rmse_log.mean())
approx_multiplier_error


np.float64(1.615946679749969)

## Cross-Validation Results

5-fold cross-validation was performed using Ridge Regression on the log-transformed target variable.

The model achieved:

- **Mean log-RMSE:** 0.480  
- **Standard deviation:** 0.014  

The low standard deviation across folds indicates stable performance and low sensitivity to data splits.

Interpreting the log-scale error, the model exhibits an approximate multiplicative error of 1.62, meaning predictions are typically within ±62% of actual rental values.

These results confirm that Ridge Regression generalizes consistently across different subsets of the data and supports its selection as the final model.
